In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
from datetime import datetime
from scipy.stats import linregress


In [9]:
visitor_path = "Data/changes-visitors-covid.csv"
covid_path = "Data/covid-data.csv"
debt_path = "Data/debt-relief-covid.csv"
income_path = "Data/income-support-covid.csv"
face_path = "Data/face-covering-policies-covid.csv"
travel_path = "Data/international-travel-covid.csv"
school_path = "Data/school-closures-covid.csv"


In [10]:
visitor_data = pd.read_csv(visitor_path)
covid_data = pd.read_csv(covid_path)
debt_data = pd.read_csv(debt_path)
income_data = pd.read_csv(income_path)
face_data = pd.read_csv(face_path)
travel_data = pd.read_csv(travel_path)
school_data = pd.read_csv(school_path)


In [11]:
visitor_data.head()

,Entity,Code,Day,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,Afghanistan,AFG,2020-02-17,0.00,4.000,1.333,5.667,1.000,-6.333
1,Afghanistan,AFG,2020-02-18,0.75,4.250,0.750,6.500,1.500,-3.250
2,Afghanistan,AFG,2020-02-19,0.40,4.000,0.800,5.200,1.400,-1.600
3,Afghanistan,AFG,2020-02-20,0.00,3.833,0.833,5.000,1.000,-0.333
4,Afghanistan,AFG,2020-02-21,0.00,4.571,0.714,5.286,1.429,0.571


In [12]:
covid_data.location.nunique()

230

In [13]:
covid_data.location.unique().tolist()

['Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Europe',
 'European Union',


In [14]:
continent = ['North America','South America','Africa','Asia','Europe']

In [15]:
# removing continents from data frame
covid_data[~covid_data.location.isin(continent)]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96203,ZWE,Africa,Zimbabwe,2021-06-12,39852.0,164.0,97.714,1632.0,3.0,3.857,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96204,ZWE,Africa,Zimbabwe,2021-06-13,39959.0,107.0,110.000,1632.0,0.0,3.714,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96205,ZWE,Africa,Zimbabwe,2021-06-14,40077.0,118.0,119.857,1635.0,3.0,3.429,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96206,ZWE,Africa,Zimbabwe,2021-06-15,40318.0,241.0,142.429,1637.0,2.0,2.857,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


In [16]:
covid_data.info

<bound method DataFrame.info of       iso_code continent     location        date  total_cases  new_cases  \
0          AFG      Asia  Afghanistan  2020-02-24          1.0        1.0   
1          AFG      Asia  Afghanistan  2020-02-25          1.0        0.0   
2          AFG      Asia  Afghanistan  2020-02-26          1.0        0.0   
3          AFG      Asia  Afghanistan  2020-02-27          1.0        0.0   
4          AFG      Asia  Afghanistan  2020-02-28          1.0        0.0   
...        ...       ...          ...         ...          ...        ...   
96203      ZWE    Africa     Zimbabwe  2021-06-12      39852.0      164.0   
96204      ZWE    Africa     Zimbabwe  2021-06-13      39959.0      107.0   
96205      ZWE    Africa     Zimbabwe  2021-06-14      40077.0      118.0   
96206      ZWE    Africa     Zimbabwe  2021-06-15      40318.0      241.0   
96207      ZWE    Africa     Zimbabwe  2021-06-16      40556.0      238.0   

       new_cases_smoothed  total_deaths  ne

In [17]:
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [18]:
covid_data.head()
covid_df = round(covid_data.groupby(["location"])["total_cases"].max(),2)
# type(covid_df)
covid_df.dropna(inplace=True)
covid_df

location
Afghanistan        93288.0
Africa           5101570.0
Albania           132476.0
Algeria           134458.0
Andorra            13836.0
                  ...     
Vietnam            11794.0
World          177040182.0
Yemen               6867.0
Zambia            118850.0
Zimbabwe           40556.0
Name: total_cases, Length: 201, dtype: float64

In [19]:
afg_data = covid_data[covid_data.location == "Afghanistan"].describe()

In [49]:
afg_data = covid_data[covid_data.location == "Afghanistan"]
afg_data

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,AFG,Asia,Afghanistan,2021-06-12,88740.0,1024.0,1539.571,3449.0,37.0,49.286,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
475,AFG,Asia,Afghanistan,2021-06-13,89861.0,1121.0,1519.571,3527.0,78.0,54.571,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
476,AFG,Asia,Afghanistan,2021-06-14,91458.0,1597.0,1516.714,3612.0,85.0,60.714,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
477,AFG,Asia,Afghanistan,2021-06-15,93272.0,1814.0,1563.714,3683.0,71.0,61.714,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [21]:
afg_data['month_year'] = pd.to_datetime(afg_data['date']).dt.to_period('M')
afg_data.head()

<ipython-input-21-fb8835b80fff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afg_data['month_year'] = pd.to_datetime(afg_data['date']).dt.to_period('M')


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality,month_year
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02


In [22]:
afg_df = afg_data.groupby(["month_year"])[["location","total_cases","new_cases","month_year"]].max()
afg_df

,location,total_cases,new_cases,month_year
month_year,,,,
2020-02,Afghanistan,1.0,1.0,2020-02
2020-03,Afghanistan,175.0,33.0,2020-03
2020-04,Afghanistan,2127.0,187.0,2020-04
2020-05,Afghanistan,15208.0,866.0,2020-05
2020-06,Afghanistan,31507.0,915.0,2020-06
2020-07,Afghanistan,36665.0,348.0,2020-07
2020-08,Afghanistan,38159.0,108.0,2020-08
2020-09,Afghanistan,39268.0,125.0,2020-09
2020-10,Afghanistan,41425.0,141.0,2020-10


In [23]:
total_new_cases= afg_data.groupby(["month_year"])["new_cases"].sum()
total_new_cases

month_year
2020-02        1.0
2020-03      174.0
2020-04     1952.0
2020-05    13081.0
2020-06    16299.0
2020-07     5158.0
2020-08     1494.0
2020-09     1109.0
2020-10     2157.0
2020-11     4849.0
2020-12     5252.0
2021-01     3497.0
2021-02      691.0
2021-03      740.0
2021-04     3291.0
2021-05    12093.0
2021-06    21450.0
Freq: M, Name: new_cases, dtype: float64

In [43]:
total_death= afg_data.groupby(["month_year"])["total_deaths"].sum()
total_death

month_year
2020-02        0.0
2020-03       27.0
2020-04      890.0
2020-05     5007.0
2020-06    15016.0
2020-07    33518.0
2020-08    42101.0
2020-09    42987.0
2020-10    46287.0
2020-11    49133.0
2020-12    62357.0
2021-01    71838.0
2021-02    67864.0
2021-03    76227.0
2021-04    76300.0
2021-05    85386.0
2021-06    52796.0
Freq: M, Name: total_deaths, dtype: float64

In [44]:
merge_data = pd.DataFrame({"total_death":total_death, "total_new_cases": total_new_cases})
merge_data


,total_death,total_new_cases
month_year,,
2020-02,0.0,1.0
2020-03,27.0,174.0
2020-04,890.0,1952.0
2020-05,5007.0,13081.0
2020-06,15016.0,16299.0
2020-07,33518.0,5158.0
2020-08,42101.0,1494.0
2020-09,42987.0,1109.0
2020-10,46287.0,2157.0


In [47]:
merge_data["year_month"] = afg_df["month_year"]
merge_data.rename("month_year", inplace=False)[source]
merge_data

TypeError: Index(...) must be called with a collection of some kind, 'idx' was passed

In [39]:
merge_afg = pd.merge(afg_df,merge_data)


ValueError: 'month_year' is both an index level and a column label, which is ambiguous.

In [ ]:
covid_data[covid_data.total_cases == 93288.000000]


In [ ]:
len(covid_data)

In [ ]:
debt_data.head()

In [ ]:
income_data.head()


In [ ]:

income_df = round(income_data.groupby(["Entity"])["income_support"].mean(),2)
income_df
income_df.sort_values(ascending = False)

In [ ]:
df = income_data.set_index("Entity")

# df = income_data.loc["Afghanistan"]
df_income = df.loc["Bulgaria"]
df_income["income_support"].sort_values(ascending = False)

In [ ]:
face_data.head()

In [ ]:
travel_data.head()

In [ ]:
school_data.head()
